In [ ]:
from airflow import DAG
from airflow.operators.empty import EmptyOperator
from airflow.providers.papermill.operators.papermill import PapermillOperator
from airflow.datasets import Dataset
from datetime import datetime

# Caminhos dos notebooks
notebook_paths = {
    "extraction": "/home/kenote_ubuntu/projetos/Airflow/notebooks/01_extraction.ipynb",
    "bronze": "/home/kenote_ubuntu/projetos/Airflow/notebooks/02_bronze_layer.ipynb",
    "silver": "/home/kenote_ubuntu/projetos/Airflow/notebooks/03_silver_layer.ipynb",
    "gold": "/home/kenote_ubuntu/projetos/Airflow/notebooks/04_gold_layer.ipynb"
}

# Dataset da camada Gold
gold_dataset = Dataset("/home/kenote_ubuntu/projetos/Airflow/data/gold/forecast_summary")

with DAG(
    dag_id="open_meteo_pipeline",
    description="⛅ Pipeline completa Open Meteo até Gold",
    schedule=None,
    start_date=datetime(2025, 4, 15),
    catchup=False,
    tags=["open-meteo", "climate", "pipeline"],
) as dag:

    start = EmptyOperator(task_id="start")

    extract = PapermillOperator(
        task_id="extract_data",
        input_nb=notebook_paths["extraction"],
        output_nb=notebook_paths["extraction"].replace(".ipynb", "_out.ipynb")
    )

    bronze = PapermillOperator(
        task_id="generate_bronze",
        input_nb=notebook_paths["bronze"],
        output_nb=notebook_paths["bronze"].replace(".ipynb", "_out.ipynb")
    )

    silver = PapermillOperator(
        task_id="generate_silver",
        input_nb=notebook_paths["silver"],
        output_nb=notebook_paths["silver"].replace(".ipynb", "_out.ipynb")
    )

    gold = PapermillOperator(
        task_id="generate_gold",
        input_nb=notebook_paths["gold"],
        output_nb=notebook_paths["gold"].replace(".ipynb", "_out.ipynb"),
        outlets=[gold_dataset],
    )

    success_marker = EmptyOperator(task_id="success_marker")

    start >> extract >> bronze >> silver >> gold >> success_marker
